In [18]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from textblob import TextBlob
import syllapy
import os
output_directory = r'C:\Users\abhis\Desktop\projects task\extracted_articles'

In [2]:
articles = {}
for filename in os.listdir(output_directory):
    if filename.endswith('.txt'):
        url_id = filename.split('.')[0]
        with open(os.path.join(output_directory, filename), 'r', encoding='utf-8') as file:
            content = file.read()
        articles[url_id] = content

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\abhis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import spacy
from textblob import TextBlob
import syllapy

In [5]:
nlp = spacy.load('en_core_web_sm')

In [6]:
def analyze_text(text):
    """
    Perform textual analysis on the given text and return various metrics.
    
    Args:
        text (str): The text to analyze.
    
    Returns:
        dict: A dictionary containing textual analysis metrics.
    """
    doc = nlp(text)
    
    # Tokenize sentences and words using spaCy
    sentences = list(doc.sents)
    words = [token.text for token in doc if token.is_alpha]
    
    # Count words and syllables
    word_count = len(words)
    syllable_count = sum(syllapy.count(word) for word in words)
    avg_word_length = sum(len(word) for word in words) / word_count if word_count > 0 else 0
    
    # Calculate TextBlob scores
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    
    # Count complex words
    stop_words = set(nlp.Defaults.stop_words)
    complex_words = [word for word in words if len(word) > 2 and word.lower() not in stop_words]
    complex_word_count = len(complex_words)
    percentage_complex_words = (complex_word_count / word_count) * 100 if word_count > 0 else 0
    
    # Calculate Fog Index
    avg_sentence_length = word_count / len(sentences) if len(sentences) > 0 else 0
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    
    # Count personal pronouns
    personal_pronouns = [word for word in words if word.lower() in ['i', 'me', 'my', 'mine', 'we', 'our', 'ours', 'us']]
    personal_pronoun_count = len(personal_pronouns)
    
    return {
        'positive_score': max(polarity, 0),
        'negative_score': min(polarity, 0),
        'polarity_score': polarity,
        'subjectivity_score': subjectivity,
        'avg_sentence_length': avg_sentence_length,
        'percentage_complex_words': percentage_complex_words,
        'fog_index': fog_index,
        'avg_words_per_sentence': word_count / len(sentences) if len(sentences) > 0 else 0,
        'complex_word_count': complex_word_count,
        'word_count': word_count,
        'syllable_per_word': syllable_count / word_count if word_count > 0 else 0,
        'personal_pronouns': personal_pronoun_count,
        'avg_word_length': avg_word_length
    }

In [7]:
# Sample articles dictionary with URL_IDs and their content
articles = {
    'bctech2073': """
    Title: Future of AI in Technology
    The rise of artificial intelligence (AI) is transforming industries. AI's applications are vast, ranging from healthcare to finance. In healthcare, AI assists in diagnosing diseases early and predicting patient outcomes. In finance, it is used for fraud detection and algorithmic trading. The future of AI looks promising with advancements in machine learning and natural language processing. However, ethical considerations and data privacy concerns must be addressed to harness AI's full potential responsibly.
    """,
    
    'bctech2074': """
    Title: The Impact of Climate Change on Coastal Cities
    Coastal cities are facing increasing threats from climate change. Rising sea levels, caused by melting polar ice and thermal expansion of seawater, pose significant risks. Storm surges and flooding are becoming more frequent and severe, affecting infrastructure and communities. Mitigation strategies include constructing sea walls, improving drainage systems, and developing sustainable urban planning practices. The adaptation efforts are critical to minimize damage and ensure the resilience of coastal cities.
    """,
    
    'bctech2075': """
    Title: Innovations in Renewable Energy
    Renewable energy technologies are advancing rapidly, offering sustainable solutions to meet global energy demands. Solar power, wind energy, and hydroelectricity are leading the way. Innovations in photovoltaic cells, wind turbine designs, and energy storage systems are enhancing efficiency and reducing costs. The shift towards renewables is driven by the need to combat climate change and reduce reliance on fossil fuels. As technology evolves, the integration of renewables into energy grids will become more seamless and widespread.
    """,
    
    'bctech2076': """
    Title: Trends in Modern Education Technology
    Education technology is evolving with the integration of digital tools and platforms. Online learning, interactive simulations, and virtual classrooms are becoming common. These technologies offer personalized learning experiences and access to a global knowledge base. The use of artificial intelligence and data analytics in education helps track student progress and tailor educational content. While technology enhances learning opportunities, challenges such as digital divide and data security must be addressed.
    """,
    
    'bctech2077': """
    Title: The Role of Big Data in Business Strategy
    Big data analytics is revolutionizing business strategies by providing actionable insights from vast amounts of data. Companies use big data to understand consumer behavior, optimize operations, and drive innovation. Advanced analytics tools help in predictive modeling and decision-making processes. The effective use of big data requires robust data management practices and skilled professionals to interpret complex datasets. Leveraging big data can lead to competitive advantages and improved business outcomes.
    """
}

# This dictionary contains sample articles with URL_IDs and corresponding content.

In [8]:
results = []

for url_id, content in articles.items():
    analysis = analyze_text(content)
    results.append({'URL_ID': url_id, **analysis})

# Convert results to DataFrame
results_df = pd.DataFrame(results)

In [15]:
results_df

,URL_ID,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,percentage_complex_words,fog_index,avg_words_per_sentence,complex_word_count,word_count,syllable_per_word,personal_pronouns,avg_word_length
0,bctech2073,0.050000,0.00,0.050000,0.559091,13.0,56.410256,27.764103,13.0,44,78,1.897436,0,5.794872
1,bctech2074,0.191667,0.00,0.191667,0.510714,15.6,73.076923,35.470769,15.6,57,78,1.935897,0,6.141026
2,bctech2075,0.200000,0.00,0.200000,0.200000,16.2,67.901235,33.640494,16.2,55,81,2.074074,0,5.975309
3,bctech2076,0.000000,-0.08,-0.080000,0.440000,15.0,68.000000,33.200000,15.0,51,75,2.226667,0,6.400000
4,bctech2077,0.120000,0.00,0.120000,0.380000,15.8,77.215190,37.206076,15.8,61,79,2.113924,0,6.063291


In [16]:
csv_file_path = r'C:\Users\abhis\Desktop\projects task\analysis_results.csv'
excel_file_path = r'C:\Users\abhis\Desktop\projects task\analysis_results.xlsx'

In [17]:
results_df.to_csv(csv_file_path, index=False)
results_df.to_excel(excel_file_path, index=False)

In [20]:
directory = 'C:/Users/abhis/Desktop/projects task/extracted_articles'

In [21]:
articles = {}
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            articles[filename] = file.read()

In [22]:
results = []
for url_id, content in articles.items():
    analysis = analyze_text(content)
    results.append({'URL_ID': url_id, **analysis})

# Convert results to DataFrame
results_df = pd.DataFrame(results)

In [23]:
results_df

,URL_ID,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,percentage_complex_words,fog_index,avg_words_per_sentence,complex_word_count,word_count,syllable_per_word,personal_pronouns,avg_word_length
0,bctech2011.txt,0.083778,0.0,0.083778,0.393588,18.985507,69.694656,35.472065,18.985507,1826,2620,2.162214,3,6.327099
1,bctech2012.txt,0.141558,0.0,0.141558,0.377201,11.836735,74.655172,34.596763,11.836735,433,580,2.410345,1,7.158621
2,bctech2013.txt,0.142706,0.0,0.142706,0.458439,16.363636,75.000000,36.545455,16.363636,540,720,2.248611,2,6.656944
3,bctech2014.txt,0.082168,0.0,0.082168,0.255944,10.448276,76.897690,34.938386,10.448276,466,606,2.377888,2,6.986799
4,bctech2015.txt,0.101338,0.0,0.101338,0.391190,21.343750,65.446559,34.716124,21.343750,447,683,2.156662,2,6.439239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,bctech2153.txt,0.084077,0.0,0.084077,0.389070,36.000000,66.452991,40.981197,36.000000,622,936,2.057692,4,5.947650
143,bctech2154.txt,0.058422,0.0,0.058422,0.253377,23.672131,62.257618,34.371900,23.672131,899,1444,1.851801,10,5.439058
144,bctech2155.txt,0.080068,0.0,0.080068,0.328829,27.666667,58.433735,34.440161,27.666667,291,498,1.887550,15,5.652610
145,bctech2156.txt,0.030114,0.0,0.030114,0.440909,63.750000,82.352941,58.441176,63.750000,210,255,2.329412,0,6.878431


In [24]:
output_file = 'C:/Users/abhis/Desktop/projects task/textual_analysis_results.xlsx'
results_df.to_excel(output_file, index=False)